### <center> TP1 Airbnb Data cleaning

In [180]:
import pandas as pd
import numpy as np
pd.set_option('display.max_rows', None)

# Reading the file

In [181]:
boston = pd.read_csv("data/boston.csv")

cambridge = pd.read_csv("data/cambridge.csv")

df = boston
#df = pd.concat([boston, cambridge])

In [182]:
df.shape

(3507, 106)

# Deleting columns

In [183]:
dtypes = {
    "int64": "int",
    "int32": "int",
    "bool": "bool",
    "float64": "float",
    "object": "str"
}

info_list = []

for col in df.columns:
    
    distinct = df[col].nunique()
    missing = df[col].isna().sum()
    dtype = df[col].dtype.name
    
    #  check if the column is a duplicate of another column
    is_duplicate = ''
    for col2 in df.columns:
        if col != col2 and df[col].equals(df[col2]):
            is_duplicate = True
            break
    
    # check if the column contains id or url in the name
    is_irrelevant = ''
    if "id" in col or "url" in col:
        is_irrelevant = True
    
    
    # get a sample value
    sample = df[col].sample(1).values[0]
    
    info_list.append({"column": col, "distinct": distinct, "missing": missing,
                      "dtype": dtypes[dtype], "sample": sample, "is_duplicate": is_duplicate,"is_irrelevant": is_irrelevant})

info = pd.DataFrame(info_list)
info

,column,distinct,missing,dtype,sample,is_duplicate,is_irrelevant
0,id,3507,0,int,7309178,,True
1,listing_url,3507,0,str,https://www.airbnb.com/rooms/34020828,,True
2,scrape_id,1,0,int,20191204162830,,True
3,last_scraped,2,0,str,2019-12-04,True,
4,name,3408,0,str,Comfy welcoming room in Boston by T,,
5,summary,2527,77,str,Private Bedroom Suite on Jamaica Plain triple ...,,
6,space,2043,667,str,You will be occupying our guest bedroom with p...,,
7,description,2766,42,str,"Astounding 2 story walk-up Studio (two Room, o...",,
8,experiences_offered,1,0,str,none,,
9,neighborhood_overview,1451,951,str,Brighton is a very quiet and at the same time ...,,


### the columns that have missing values > 50% of the total number of rows

In [184]:
missing_cols = info[info["missing"] > 0.5 * df.shape[0]]["column"].values
missing_cols = [str(col) for col in missing_cols]
missing_cols

['thumbnail_url',
 'medium_url',
 'xl_picture_url',
 'host_acceptance_rate',
 'neighbourhood_group_cleansed',
 'square_feet',
 'weekly_price',
 'monthly_price']

### some text columns could be converted to categorical columns
we select only the text columns that have less 10 categories (distinct values) 

In [185]:
text_cols = info[info["dtype"] == "str"]["column"].values
is_text = []

for col in text_cols:
    #print(col, df[col].nunique())
    if df[col].nunique() > 10:
        is_text.append(col)
is_text

['listing_url',
 'name',
 'summary',
 'space',
 'description',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'picture_url',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_rate',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_verifications',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'city',
 'zipcode',
 'smart_location',
 'property_type',
 'amenities',
 'price',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 'cleaning_fee',
 'extra_people',
 'calendar_updated',
 'first_review',
 'last_review',
 'license']

### duplicate columns

In [186]:
duplicate_cols = info[info["is_duplicate"] == True]["column"].values
duplicate_cols

array(['last_scraped', 'thumbnail_url', 'medium_url', 'xl_picture_url',
       'host_acceptance_rate', 'host_listings_count',
       'host_total_listings_count', 'neighbourhood_group_cleansed',
       'calendar_last_scraped'], dtype=object)

### the columns that have less than 2 distinct values

In [187]:
not_distinct_cols = info[info["distinct"] < 2]["column"].values
not_distinct_cols

array(['scrape_id', 'experiences_offered', 'thumbnail_url', 'medium_url',
       'xl_picture_url', 'host_acceptance_rate',
       'neighbourhood_group_cleansed', 'market', 'country_code',
       'country', 'has_availability', 'jurisdiction_names',
       'is_business_travel_ready'], dtype=object)

### the columns that contain id or url in the name

In [188]:
is_irrelevant_cols = info[info["is_irrelevant"] == True]["column"].values
is_irrelevant_cols

array(['id', 'listing_url', 'scrape_id', 'thumbnail_url', 'medium_url',
       'picture_url', 'xl_picture_url', 'host_id', 'host_url',
       'host_thumbnail_url', 'host_picture_url', 'host_identity_verified'],
      dtype=object)

### any extra columns we want to remove

In [189]:
extra_cols = ["state"]
df["state"].value_counts()

state
MA             3506
Allston, Ma       1
Name: count, dtype: int64

In [190]:
delete_cols = np.concatenate([missing_cols, duplicate_cols, 
                              not_distinct_cols, is_irrelevant_cols, 
                              is_text , extra_cols])

In [191]:
info = info[~info["column"].isin(delete_cols)]
info = info.drop(columns=["is_duplicate", "is_irrelevant"])
info = info.sort_values(by="missing", ascending=False)
info = info.reset_index(drop=True)
info

,column,distinct,missing,dtype,sample
0,review_scores_checkin,8,697,float,NaN
1,review_scores_value,8,696,float,NaN
2,review_scores_location,8,696,float,10.0
3,review_scores_accuracy,9,696,float,10.0
4,review_scores_cleanliness,8,695,float,9.0
5,review_scores_rating,42,694,float,NaN
6,review_scores_communication,9,694,float,9.0
7,reviews_per_month,707,684,float,7.89
8,host_response_time,4,409,str,within a day
9,host_has_profile_pic,2,15,str,t


In [192]:
df= df.drop(delete_cols, axis=1)
df.shape

(3507, 44)

# Handling missing values

In [193]:
missing = df.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing_percentage = (missing / df.shape[0]) * 100
missing = pd.DataFrame(
    {'missing': missing, 'Percentage %': round(missing_percentage, 2)})
missing

,missing,Percentage %
review_scores_checkin,697,19.87
review_scores_accuracy,696,19.85
review_scores_location,696,19.85
review_scores_value,696,19.85
review_scores_cleanliness,695,19.82
review_scores_rating,694,19.79
review_scores_communication,694,19.79
reviews_per_month,684,19.50
host_response_time,409,11.66
host_is_superhost,15,0.43


### filling the columns that have few missing values with the average

In [194]:
for col in missing.index:
    
    if missing.loc[col, "Percentage %"] < 5:
        
        # if it's a str column, fill with the most common value
        if df[col].dtype == "object":
            value = df[col].mode()[0]
            print(
                f"filling {col} with the most common value {value}")
            df[col] = df[col].fillna(df[col].mode()[0])
        
        # if it's a numerical column, fill with the most common value

        if df[col].dtype == "int" or df[col].dtype == "float":
            value = round(df[col].mean())
            print(
                f"filling {col} with the mean value {value}")
            
            df[col] = df[col].fillna(df[col].mean())
            


filling host_is_superhost with the most common value f
filling host_has_profile_pic with the most common value t
filling beds with the mean value 2
filling bedrooms with the mean value 1
filling bathrooms with the mean value 1


In [195]:
# change the host_response_time Nones to unknown
df["host_response_time"] = df["host_response_time"].fillna("unknown")

In [196]:
missing = df.isna().sum()
missing = missing[missing > 0].sort_values(ascending=False)
missing


review_scores_checkin          697
review_scores_accuracy         696
review_scores_location         696
review_scores_value            696
review_scores_cleanliness      695
review_scores_rating           694
review_scores_communication    694
reviews_per_month              684
dtype: int64

### Fixing boolean columns

In [197]:
# if the column contains "t","f" we can convert it to bool
bool_cols = []
for col in df.columns:
    if df[col].nunique() == 2 and "t" in df[col].unique() and "f" in df[col].unique():
        bool_cols.append(col)

bool_cols

['host_is_superhost',
 'host_has_profile_pic',
 'is_location_exact',
 'requires_license',
 'instant_bookable',
 'require_guest_profile_picture',
 'require_guest_phone_verification']

In [198]:
# convert the columns to bool
for col in bool_cols:
    df[col] = df[col].map({"t": True, "f": False})
    df[col] = df[col].astype("bool")

### Handling Categorical data

In [199]:
# for the rest of the str columns, we can convert them to categorical
categorical_cols = info[info["dtype"] == "str"]["column"].values
categorical_cols = [col for col in categorical_cols if col not in bool_cols]
categorical_cols

['host_response_time', 'cancellation_policy', 'room_type', 'bed_type']

#### one hot encode the categorical columns

In [200]:
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
df.shape

(3507, 56)

#### removing highly correlated columns > 60%

In [201]:

correlation = df.corr()
correlation = correlation.abs().unstack()
correlation = correlation.sort_values(ascending=False)
correlation = correlation[correlation > 0.60]
correlation = correlation[correlation < 1]
correlation = pd.DataFrame(correlation).reset_index()
correlation.columns = ["column 1", "column 2", "correlation"]
correlation = correlation.drop_duplicates(subset="correlation")
correlation



,column 1,column 2,correlation
0,maximum_maximum_nights,maximum_nights_avg_ntm,0.999996
2,minimum_maximum_nights,maximum_nights_avg_ntm,0.996293
4,minimum_maximum_nights,maximum_maximum_nights,0.996148
6,calculated_host_listings_count,calculated_host_listings_count_entire_homes,0.987408
8,minimum_nights_avg_ntm,maximum_minimum_nights,0.985635
10,availability_60,availability_90,0.972321
12,minimum_minimum_nights,minimum_nights,0.968473
14,minimum_nights_avg_ntm,minimum_nights,0.961486
16,minimum_minimum_nights,minimum_nights_avg_ntm,0.947199
18,maximum_minimum_nights,minimum_nights,0.938598


In [202]:
highly_correlated_cols = correlation["column 2"].unique()
df.drop(highly_correlated_cols, axis=1, inplace=True)
df.shape

(3507, 28)

#### 

In [203]:
df.to_excel("data/cleaned_data.xlsx", index=False)